In [1]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon.text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 14.5 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB

In [2]:
!pip install autogluon.tabular

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.2/292.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.3/218.3 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: autogluon.common
    Found existing installation: autogluon.common 0.6.2
    Uninstalling autogluon.common-0.6.2:
      Successfully uninstalled autogluon.common-0.6.2
  Attempting uninstall: autogluon.features
    Found existing installation: autogluon.features 0.6.2
    Uninstalling autogluon.features-0.6.2:
      Successfully uninstalled autogluon.features-0.6.2
  Attempting uninstall: autogluon.core
    Found existing installation: autogluon.core 0.6.2
    Uninstalling autogluon.core-0.6.2:
      Successfully uninstalled autogluon.core-0.6.2
ERR

In [3]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # 행운의 seed

In [30]:
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')

In [31]:
# 독립변수로 설정할 train_x에서는 종속변수를 제거합니다. 또한 분석에 활용하지 않는 ID 데이터를 제거합니다.
train_x = train.drop(['ID','Calories_Burned'], axis = 1)
# train_y 변수를 종속변수로 사용하기 위해 Calories_Burned 데이터를 지정하였습니다.
train_y = train['Calories_Burned']

# train_x 데이터와 마찬가지로 분석에 활용하지 않는 ID 데이터를 제거합니다.
test_x = test.drop('ID', axis = 1)

ordinal_features = ['Weight_Status', 'Gender']

for feature in ordinal_features:
    le = LabelEncoder()
    le = le.fit(train_x[feature])
    train_x[feature] = le.transform(train_x[feature])

    # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
    # 따라서 test 데이터를 바로 변형시키지 않고 고윳값을 확인후 test 데이터를 변환합니다.
    for label in np.unique(test_x[feature]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[feature] = le.transform(test_x[feature])

In [32]:
poly = PolynomialFeatures()
train_poly = pd.DataFrame(poly.fit_transform(train_x))
test_poly = pd.DataFrame(poly.fit_transform(test_x))

In [39]:
train_poly['target'] = train_y

In [43]:
train_data = TabularDataset(train_poly)
test_data = TabularDataset(test_poly)

In [44]:
label = 'target'
time_limit=3600

In [ ]:
#excluded_model_types = ['CAT']
predictor = TabularPredictor(
    label=label, problem_type='regression', eval_metric='root_mean_squared_error',
).fit(train_data, 
      presets='best_quality', 
      num_stack_levels=3,
      #excluded_model_types = excluded_model_types,
      time_limit=time_limit, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20230415_153906/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "AutogluonModels/ag-20230415_153906/"
AutoGluon Version:  0.7.0
Python Version:     3.9.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Dec 10 16:00:40 UTC 2022
Train Data Rows:    7500
Train Data Columns: 55
Label Column: target
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    10705.79 MB
	Train Data (Original)  Memory Usage: 3.3 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Not

[1000]	valid_set's rmse: 1.56495
[2000]	valid_set's rmse: 1.46633
[3000]	valid_set's rmse: 1.43146
[4000]	valid_set's rmse: 1.41569
[5000]	valid_set's rmse: 1.40593
[6000]	valid_set's rmse: 1.40089
[7000]	valid_set's rmse: 1.39708
[8000]	valid_set's rmse: 1.39469
[9000]	valid_set's rmse: 1.39171
[10000]	valid_set's rmse: 1.39084
[1000]	valid_set's rmse: 1.78836
[2000]	valid_set's rmse: 1.69889
[3000]	valid_set's rmse: 1.66987
[4000]	valid_set's rmse: 1.65883
[5000]	valid_set's rmse: 1.65131
[6000]	valid_set's rmse: 1.64658
[7000]	valid_set's rmse: 1.64446
[8000]	valid_set's rmse: 1.64264
[9000]	valid_set's rmse: 1.64149
[10000]	valid_set's rmse: 1.64068
[1000]	valid_set's rmse: 1.32169
[2000]	valid_set's rmse: 1.26204
[3000]	valid_set's rmse: 1.24186
[4000]	valid_set's rmse: 1.23376
[5000]	valid_set's rmse: 1.22771
[6000]	valid_set's rmse: 1.22412
[7000]	valid_set's rmse: 1.22212
[8000]	valid_set's rmse: 1.22065
[9000]	valid_set's rmse: 1.21971
[10000]	valid_set's rmse: 1.2189
[1000]	v

	-1.3964	 = Validation score   (-root_mean_squared_error)
	228.3s	 = Training   runtime
	9.92s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 941.01s of the 3341.2s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


[1000]	valid_set's rmse: 1.59065
[2000]	valid_set's rmse: 1.55259
[3000]	valid_set's rmse: 1.54642
[4000]	valid_set's rmse: 1.54157
[5000]	valid_set's rmse: 1.54008
[6000]	valid_set's rmse: 1.53929
[7000]	valid_set's rmse: 1.53861
[8000]	valid_set's rmse: 1.53832
[9000]	valid_set's rmse: 1.53817
[10000]	valid_set's rmse: 1.53814


In [12]:
model_to_use = predictor.get_model_best()

In [14]:
model_pred = predictor.predict(test_data, model=model_to_use)

KeyError: ignored